# Step 1. Open the data file and study the general information

## Project description
We work for the online store Ice, which sells video games all over the world.
We need to identify patterns that determine whether a game succeeds or not. This will allow us to spot potential big winners and plan advertising campaigns.


## Import

In [417]:
import pandas as pd
import numpy as np
import chart_studio.plotly as py
import seaborn as sns
import plotly.express as px
from scipy import stats as st
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport
import sidetable


%matplotlib inline

## Load data

In [418]:
try:
    df_games_raw = pd.read_csv('games.csv')
except:
    df_games_raw = pd.read_csv('/datasets/games.csv')

## Explore initial data

In [419]:
print('General info about the data')
print(df_games_raw.info())
print()

print('Five first rows')
print(df_games_raw.head())
print()

print('Description of the numerical columns')
print(df_games_raw.describe())
print()

print('Description of the textual columns')
print(df_games_raw.describe(include=object))
print()

General info about the data
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16713 non-null  object 
 1   Platform         16715 non-null  object 
 2   Year_of_Release  16446 non-null  float64
 3   Genre            16713 non-null  object 
 4   NA_sales         16715 non-null  float64
 5   EU_sales         16715 non-null  float64
 6   JP_sales         16715 non-null  float64
 7   Other_sales      16715 non-null  float64
 8   Critic_Score     8137 non-null   float64
 9   User_Score       10014 non-null  object 
 10  Rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB
None

Five first rows
                       Name Platform  Year_of_Release         Genre  NA_sales  \
0                Wii Sports      Wii           2006.0        Sports     41.36   
1         Super Mario Bros.  

In [420]:
df_games_raw.stb.missing(style=True)

,missing,total,percent
Critic_Score,"8,578","16,715",51.32%
Rating,"6,766","16,715",40.48%
User_Score,"6,701","16,715",40.09%
Year_of_Release,269,"16,715",1.61%
Name,2,"16,715",0.01%
Genre,2,"16,715",0.01%
Platform,0,"16,715",0.00%
NA_sales,0,"16,715",0.00%
EU_sales,0,"16,715",0.00%
JP_sales,0,"16,715",0.00%


### Notes on explore initial data

The data contains the following columns:
- Name
- Platform
- Year_of_Release
- Genre
- NA_sales (North American sales in USD million)
- EU_sales (sales in Europe in USD million)
- JP_sales (sales in Japan in USD million)
- Other_sales (sales in other countries in USD million)
- Critic_Score (maximum of 100)
- User_Score (maximum of 10)
- Rating (ESRB)

In the data we have 16,715 entries. Only in the Platform column and all 4 sales column (NA, EU, JP and others) all rows with values. 
In  Name and Genre we have 2 missing values.
In Year_of_Release we have 269 missing values.
In Critic_Score we have 8,578 missing values - this is more than 50% !
In Rating we have 6,766 missing values. And in User_Score we have 6,701 missing values.

From the description of the numerical columns we can see that for the sales column we have large amount of 0 values in the columns. We know that because the 1st quartile these column is 0. 

From the description of the textual columns we see that in User_Score column there is a mixture of numerical scores and text values of TBD (to be determined).

The Year of Release type should be int






# Step 2. Prepare the data

## Replace the column names (make them lowercase)

In [421]:
# rename Pandas columns to lower case 
df_games_raw.columns= df_games_raw.columns.str.lower()
df_games_raw.columns

Index(['name', 'platform', 'year_of_release', 'genre', 'na_sales', 'eu_sales',
       'jp_sales', 'other_sales', 'critic_score', 'user_score', 'rating'],
      dtype='object')

done

## Treating missing values

### name

In [422]:
df_games_raw[df_games_raw['name'].isnull()]

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
659,NaN,GEN,1993.0,NaN,1.78,0.53,0.00,0.08,NaN,NaN,NaN
14244,NaN,GEN,1993.0,NaN,0.00,0.00,0.03,0.00,NaN,NaN,NaN


Looks like someone forgot to add these names. We will fill them with 'unknown'

In [423]:
df_games = df_games_raw
df_games['name'] = df_games['name'].fillna('unknown')
df_games[df_games['name'].isnull()]

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating


done

### genere

In [424]:
df_games[df_games['genre'].isnull()]

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
659,unknown,GEN,1993.0,NaN,1.78,0.53,0.00,0.08,NaN,NaN,NaN
14244,unknown,GEN,1993.0,NaN,0.00,0.00,0.03,0.00,NaN,NaN,NaN


It's the same place where we have missing values in the name. Also many other places here with no values. We will remove these rows since there is nothing to study from them

In [425]:
df_games.dropna(subset=['genre'] ,inplace=True)
df_games[df_games['genre'].isnull()]

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating


done

### year_of_release

Add new column for duplicate game name

In [426]:
df_games['duplicate_name'] = df_games.duplicated(
    subset='name', keep=False
)

Check if we can restore rating and year of release by the names for games with duplicates 

In [427]:
df_games.query('duplicate_name').sort_values(by='name').sort_values(by='name')[1000:1020]

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,duplicate_name
14970,Captain Morgane and the Golden Turtle,PC,2012.0,Adventure,0.00,0.02,0.00,0.00,54.0,6.2,NaN,True
16359,Captain Morgane and the Golden Turtle,Wii,2012.0,Adventure,0.00,0.01,0.00,0.00,NaN,tbd,NaN,True
15225,Captain Morgane and the Golden Turtle,PS3,2012.0,Adventure,0.00,0.02,0.00,0.00,NaN,tbd,NaN,True
16530,Carmageddon: Max Damage,PS4,2016.0,Action,0.01,0.00,0.00,0.00,51.0,5.5,M,True
15456,Carmageddon: Max Damage,XOne,2016.0,Action,0.01,0.01,0.00,0.00,52.0,7.1,M,True
15050,Carmen Sandiego: The Secret of the Stolen Drums,XB,2004.0,Action,0.02,0.01,0.00,0.00,53.0,tbd,E,True
12851,Carmen Sandiego: The Secret of the Stolen Drums,PS2,2004.0,Action,0.03,0.02,0.00,0.01,53.0,tbd,E,True
15310,Carmen Sandiego: The Secret of the Stolen Drums,GC,2004.0,Action,0.02,0.00,0.00,0.00,57.0,tbd,E,True
840,Carnival Games,DS,2008.0,Misc,1.21,0.63,0.00,0.19,48.0,3.3,E,True
294,Carnival Games,Wii,2007.0,Misc,2.12,1.47,0.05,0.42,56.0,6,E,True


We see that for games that was released in multiple platforms the year_of_release and rating is same for all occurrence. So we can use the cases where there is information to complete the missings.  

In [428]:
df_games.stb.missing(style=True)

,missing,total,percent
critic_score,"8,576","16,713",51.31%
rating,"6,764","16,713",40.47%
user_score,"6,699","16,713",40.08%
year_of_release,269,"16,713",1.61%
name,0,"16,713",0.00%
platform,0,"16,713",0.00%
genre,0,"16,713",0.00%
na_sales,0,"16,713",0.00%
eu_sales,0,"16,713",0.00%
jp_sales,0,"16,713",0.00%


In [429]:
dict_of_name_and_year = dict(zip(df_games.dropna().name,df_games.dropna().year_of_release))
df_games['year_of_release'] = df_games['year_of_release'].fillna(df_games['name'].map(dict_of_name_and_year))
df_games.stb.missing(style=True)

,missing,total,percent
critic_score,"8,576","16,713",51.31%
rating,"6,764","16,713",40.47%
user_score,"6,699","16,713",40.08%
year_of_release,167,"16,713",1.00%
name,0,"16,713",0.00%
platform,0,"16,713",0.00%
genre,0,"16,713",0.00%
na_sales,0,"16,713",0.00%
eu_sales,0,"16,713",0.00%
jp_sales,0,"16,713",0.00%


We managed to fill 102 cells.
In all the rest we will put 0 to not interfere with changing type

### rating

In [430]:
dict_of_name_and_rating = dict(zip(df_games.dropna().name,df_games.dropna().rating))
df_games['rating'] = df_games['rating'].fillna(df_games['name'].map(dict_of_name_and_rating))
df_games.stb.missing(style=True)

,missing,total,percent
critic_score,"8,576","16,713",51.31%
user_score,"6,699","16,713",40.08%
rating,"6,403","16,713",38.31%
year_of_release,167,"16,713",1.00%
name,0,"16,713",0.00%
platform,0,"16,713",0.00%
genre,0,"16,713",0.00%
na_sales,0,"16,713",0.00%
eu_sales,0,"16,713",0.00%
jp_sales,0,"16,713",0.00%


We managed to fill 300 cells. 

Now we will fill in year_of_release with nan the value 0 to help us in the future analysis. And change type to int

In [431]:
df_games['year_of_release'] = df_games['year_of_release'].fillna(0).astype(int)

### tbd in user_score

In [432]:
number = df_games.query('user_score =="tbd"').shape[0]
f'We have {number} rows with tbd.'

'We have 2424 rows with tbd.'

In [433]:
df_games_before_2000 =  df_games.query('year_of_release < 2000')
df_games_after_2000 =  df_games.query('year_of_release >= 2000')

In [434]:
df_games.stb.missing(style=True)

,missing,total,percent
critic_score,"8,576","16,713",51.31%
user_score,"6,699","16,713",40.08%
rating,"6,403","16,713",38.31%
name,0,"16,713",0.00%
platform,0,"16,713",0.00%
year_of_release,0,"16,713",0.00%
genre,0,"16,713",0.00%
na_sales,0,"16,713",0.00%
eu_sales,0,"16,713",0.00%
jp_sales,0,"16,713",0.00%


In [435]:
df_games_before_2000.stb.missing(style=True)

,missing,total,percent
critic_score,"1,980","2,141",92.48%
user_score,"1,954","2,141",91.27%
rating,"1,881","2,141",87.86%
name,0,"2,141",0.00%
platform,0,"2,141",0.00%
year_of_release,0,"2,141",0.00%
genre,0,"2,141",0.00%
na_sales,0,"2,141",0.00%
eu_sales,0,"2,141",0.00%
jp_sales,0,"2,141",0.00%


In [436]:
df_games_after_2000.stb.missing(style=True)

,missing,total,percent
critic_score,"6,596","14,572",45.26%
user_score,"4,745","14,572",32.56%
rating,"4,522","14,572",31.03%
name,0,"14,572",0.00%
platform,0,"14,572",0.00%
year_of_release,0,"14,572",0.00%
genre,0,"14,572",0.00%
na_sales,0,"14,572",0.00%
eu_sales,0,"14,572",0.00%
jp_sales,0,"14,572",0.00%


We see that before the year 2000 there was about 90% missing from critic_score, user_score and rating

Let's see in each year how many values we have

In [437]:
df_games.groupby(
    by='year_of_release'
).count()[['name', 'rating', 'critic_score', 'user_score']]

,name,rating,critic_score,user_score
year_of_release,,,,
0,167,89,65,86
1980,9,1,0,0
1981,46,1,0,0
1982,36,1,0,0
1983,17,0,0,0
1984,14,0,0,0
1985,14,2,1,1
1986,21,0,0,0
1987,16,0,0,0


We see that for all 3 there is a big drop of entries before 2000 and before 1994 there was almost no entries in these fields

Now as we approaching the big 3. 
- critic_score with 51% missing that makes 8578 missing values
- user_score with 40% missing that makes 6701 missing values
- rating with 40% missing that makes 6766 missing values

from scrolling over the data we noticed that in lot of places where 1 or sometimes all of these columns where missing it was data from long time ago. In the previous millennia.
We will create data for entries before 2000 and check if before that year the rate of missing values was higher.

In [438]:
df_games.sample(10)

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,duplicate_name
1012,The Elder Scrolls Online,PS4,2015,Role-Playing,0.67,0.79,0.00,0.29,NaN,NaN,M,True
3106,Wario: Master of Disguise,DS,2007,Platform,0.28,0.02,0.33,0.03,60.0,7.4,E10+,False
15754,Virtua Quest,PS2,0,Role-Playing,0.01,0.01,0.00,0.00,53.0,7.6,T,True
16131,Kaitou Rousseau,DS,2006,Action,0.00,0.00,0.01,0.00,NaN,NaN,NaN,False
102,Need for Speed Underground,PS2,2003,Racing,3.27,2.83,0.08,1.02,85.0,8.6,E,True
7501,Beyblade G-Revolution,GBA,2004,Action,0.15,0.05,0.00,0.00,40.0,7.8,E,False
10579,Olympic Summer Games: Atlanta 1996,PS,1996,Sports,0.06,0.04,0.00,0.01,NaN,NaN,NaN,False
1983,Imagine: Animal Doctor,DS,2007,Simulation,0.45,0.48,0.00,0.11,NaN,7.4,E,False
4985,FIFA 14,Wii,2013,Sports,0.00,0.36,0.00,0.02,NaN,4.3,E,True
15925,Zoey 101: Field Trip Fiasco,DS,2007,Action,0.02,0.00,0.00,0.00,31.0,tbd,E,False


In [439]:
df_games_group_name = df_games.groupby(
    by=['name']
).count().nlargest(10, 'platform')


In [440]:
ten_most_version_games  = df_games_group_name.index.get_level_values('name').to_list()

In [441]:
for game in ten_most_version_games:
    print(game)
    print(df_games.loc[df_games['name'] == game].drop(
        labels=['na_sales', 'eu_sales', 'jp_sales', 'other_sales', 'genre'], axis=1))
    print()

Need for Speed: Most Wanted
                              name platform  year_of_release  critic_score  \
253    Need for Speed: Most Wanted      PS2             2005          82.0   
523    Need for Speed: Most Wanted      PS3             2012           NaN   
1190   Need for Speed: Most Wanted     X360             2012          83.0   
1591   Need for Speed: Most Wanted     X360             2005          83.0   
1998   Need for Speed: Most Wanted       XB             2005          83.0   
2048   Need for Speed: Most Wanted      PSV             2012           NaN   
3581   Need for Speed: Most Wanted       GC             2005          80.0   
5972   Need for Speed: Most Wanted       PC             2005          82.0   
6273   Need for Speed: Most Wanted     WiiU             2013           NaN   
6410   Need for Speed: Most Wanted       DS             2005          45.0   
6473   Need for Speed: Most Wanted      GBA             2005           NaN   
11715  Need for Speed: Most Wanted  

In [442]:
df_games.loc[df_games['name'] == 'Need for Speed: Most Wanted']

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,duplicate_name
253,Need for Speed: Most Wanted,PS2,2005,Racing,2.03,1.79,0.08,0.47,82.0,9.1,T,True
523,Need for Speed: Most Wanted,PS3,2012,Racing,0.71,1.46,0.06,0.58,NaN,NaN,T,True
1190,Need for Speed: Most Wanted,X360,2012,Racing,0.62,0.78,0.01,0.15,83.0,8.5,T,True
1591,Need for Speed: Most Wanted,X360,2005,Racing,1.00,0.13,0.02,0.10,83.0,8.5,T,True
1998,Need for Speed: Most Wanted,XB,2005,Racing,0.53,0.46,0.00,0.05,83.0,8.8,T,True
2048,Need for Speed: Most Wanted,PSV,2012,Racing,0.33,0.45,0.01,0.22,NaN,NaN,T,True
3581,Need for Speed: Most Wanted,GC,2005,Racing,0.43,0.11,0.00,0.02,80.0,9.1,T,True
5972,Need for Speed: Most Wanted,PC,2005,Racing,0.02,0.23,0.00,0.04,82.0,8.5,T,True
6273,Need for Speed: Most Wanted,WiiU,2013,Racing,0.13,0.12,0.00,0.02,NaN,NaN,T,True
6410,Need for Speed: Most Wanted,DS,2005,Racing,0.24,0.01,0.00,0.02,45.0,6.1,E,True


In [443]:

df_games.loc[df_games['name'] == 'FIFA 14']

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,duplicate_name
126,FIFA 14,PS3,2013,Sports,0.78,4.24,0.07,1.37,86.0,4.3,E,True
271,FIFA 14,X360,2013,Sports,0.92,2.89,0.01,0.40,84.0,4.2,E,True
469,FIFA 14,PS4,2013,Sports,0.61,1.85,0.11,0.44,87.0,6.3,E,True
1753,FIFA 14,XOne,2013,Sports,0.41,0.66,0.00,0.09,88.0,5.8,E,True
4699,FIFA 14,PSV,2013,Sports,0.08,0.23,0.01,0.09,NaN,1.8,E,True
4809,FIFA 14,PC,2013,Sports,0.01,0.36,0.00,0.03,87.0,4.6,E,True
4985,FIFA 14,Wii,2013,Sports,0.00,0.36,0.00,0.02,NaN,4.3,E,True
6871,FIFA 14,3DS,2013,Sports,0.00,0.22,0.00,0.01,NaN,NaN,E,True
7875,FIFA 14,PSP,2013,Sports,0.00,0.15,0.00,0.04,NaN,3.4,E,True


In [444]:
df_games.loc[df_games['name'] == 'LEGO Marvel Super Heroes']

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,duplicate_name
730,LEGO Marvel Super Heroes,X360,2013,Action,1.24,0.79,0.00,0.19,80.0,7.6,E10+,True
945,LEGO Marvel Super Heroes,PS3,2013,Action,0.77,0.76,0.01,0.29,82.0,7.9,E10+,True
1139,LEGO Marvel Super Heroes,PS4,2013,Action,0.59,0.76,0.01,0.26,83.0,7.7,E10+,True
1959,LEGO Marvel Super Heroes,XOne,2013,Action,0.61,0.35,0.00,0.09,NaN,6.8,E10+,True
2313,LEGO Marvel Super Heroes,3DS,2013,Action,0.42,0.36,0.04,0.07,61.0,5.2,E10+,True
2778,LEGO Marvel Super Heroes,WiiU,2013,Action,0.31,0.35,0.02,0.06,82.0,8,E10+,True
3876,LEGO Marvel Super Heroes,PSV,2013,Action,0.11,0.29,0.00,0.11,NaN,NaN,E10+,True
5054,LEGO Marvel Super Heroes,DS,2013,Action,0.22,0.13,0.00,0.03,NaN,4.8,E10+,True
8369,LEGO Marvel Super Heroes,PC,2013,Action,0.04,0.12,0.00,0.01,78.0,8.1,E10+,True


In [445]:

df_games.loc[df_games['name'] == 'Ratatouille']

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,duplicate_name
1598,Ratatouille,DS,2007,Action,0.49,0.62,0.0,0.14,NaN,NaN,E,True
2442,Ratatouille,PS2,2007,Action,0.31,0.00,0.0,0.53,65.0,8,E,True
3069,Ratatouille,PSP,2007,Action,0.22,0.27,0.0,0.16,64.0,7.1,E,True
3902,Ratatouille,PS3,2007,Action,0.09,0.32,0.0,0.10,55.0,4.8,E,True
3948,Ratatouille,Wii,2007,Action,0.43,0.03,0.0,0.04,62.0,6.7,E,True
6398,Ratatouille,X360,2007,Action,0.23,0.02,0.0,0.02,56.0,tbd,E,True
7681,Ratatouille,GBA,2007,Action,0.14,0.05,0.0,0.00,65.0,tbd,E,True
9015,Ratatouille,GC,2007,Action,0.11,0.03,0.0,0.00,60.0,5.6,E,True
14514,Ratatouille,PC,2007,Action,0.01,0.01,0.0,0.00,NaN,7.9,E,True


In [446]:
len(df_games['name'].unique())

11559

In [447]:
df_games.pivot_table(
    values='na_sales',
    index=('name'),
    aggfunc='count'
).sort_values(by='na_sales', ascending=False).head(10)

,na_sales
name,
Need for Speed: Most Wanted,12
LEGO Marvel Super Heroes,9
Ratatouille,9
Madden NFL 07,9
FIFA 14,9
LEGO Jurassic World,8
Madden NFL 08,8
Monopoly,8
FIFA 15,8


In [448]:
df_games.pivot_table(
    index=('name', 'platform'),
    
   
).reset_index()

,name,platform,critic_score,duplicate_name,eu_sales,jp_sales,na_sales,other_sales,year_of_release
0,Beyblade Burst,3DS,NaN,0.0,0.00,0.03,0.00,0.00,2016.0
1,Fire Emblem Fates,3DS,NaN,0.0,0.23,0.52,0.81,0.11,2015.0
2,Frozen: Olaf's Quest,3DS,NaN,1.0,0.27,0.00,0.27,0.05,2013.0
3,Frozen: Olaf's Quest,DS,NaN,1.0,0.26,0.00,0.21,0.04,2013.0
4,Haikyu!! Cross Team Match!,3DS,NaN,0.0,0.00,0.04,0.00,0.00,2016.0
...,...,...,...,...,...,...,...,...,...
16704,uDraw Studio,Wii,71.0,0.0,0.57,0.00,1.65,0.20,2010.0
16705,uDraw Studio: Instant Artist,Wii,NaN,1.0,0.09,0.00,0.06,0.02,2011.0
16706,uDraw Studio: Instant Artist,X360,54.0,1.0,0.01,0.00,0.01,0.00,2011.0
16707,wwe Smackdown vs. Raw 2006,PS2,NaN,0.0,1.02,0.00,1.57,0.41,0.0


In [449]:
df_games_before_2000 = df_games.query('year_of_release < 2005')
df_games_after_2000 = df_games.query('year_of_release >= 2005')

In [450]:
print('General info about the df_games')
print(df_games.info())
print()

print('General info about the df_games_before_2000')
print(df_games_before_2000.info())
print()

print('General info about the df_games_after_2000')
print(df_games_after_2000.info())
print()

General info about the df_games
<class 'pandas.core.frame.DataFrame'>
Int64Index: 16713 entries, 0 to 16714
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16713 non-null  object 
 1   platform         16713 non-null  object 
 2   year_of_release  16713 non-null  int32  
 3   genre            16713 non-null  object 
 4   na_sales         16713 non-null  float64
 5   eu_sales         16713 non-null  float64
 6   jp_sales         16713 non-null  float64
 7   other_sales      16713 non-null  float64
 8   critic_score     8137 non-null   float64
 9   user_score       10014 non-null  object 
 10  rating           10310 non-null  object 
 11  duplicate_name   16713 non-null  bool   
dtypes: bool(1), float64(5), int32(1), object(5)
memory usage: 1.5+ MB
None

General info about the df_games_before_2000
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5368 entries, 1 to 16713
Data columns (total

In [451]:
nan_df_games = df_games.query('user_score.isnull()', engine='python')
no_nan_df_games = df_games.query('user_score.notnull()', engine='python')
# df.query('value < 10 | value.isnull()', engine='python')
# nan_df_games

#### critic_score

In [452]:
px.histogram(data_frame=df_games, x='critic_score')

In [453]:
# df_games.loc[pd.isna(df_games['critic_score']) & pd.isna(df_games['user_score'])]
# df_games

In [454]:
# profile = ProfileReport(df_games, title='Report')
# display(profile)

In [455]:
# profile_nan = ProfileReport(nan_df_games, title='Report')
# display(profile_nan)

In [456]:
# profile_no_nan = ProfileReport(no_nan_df_games, title='Report')
# display(profile_no_nan)